In [4]:
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d
import scipy.integrate as integrate

In [35]:
hazard_rates = pd.read_csv('hazard_rates.csv')
discount_factors = pd.read_csv('discount_factors.csv')

In [25]:
time_periods_m = np.array([1/12,3/12, 6/12, 9/12, 1])
time_periods_y = np.arange(2, 31, 1)
time_periods = np.append(time_periods_m, time_periods_y)


In [32]:
discount_factors_funcs = discount_factors.apply(lambda row: interp1d(time_periods, row[1:], kind='cubic'), axis = 1)